# Filling the database
In this notebook we fill the database with three tables, one table that holds the actual routes, and two tables which serve as helpers, listing up gyms and rock climbing holds.

So after running this our database will look like this

```
         / holds - Lists the types of rock climbing holds 
climbing - gyms  - Lists gyms, with details like location, opening hours, etc.
         \ walls - Lists the actual rock climbing walls, listing ratings, difficulties, which type of holds, the gym...
```


## Imports and presetup

In [1]:
import pyodbc

import pandas as pd 
# While pandas has a builtin read_sql and read_sql_query method, it is meant for use with SQLAlchemy and would throw warnings
# So we need to write our own quick helper func
def query_to_df(query, cursor):
    cursor.execute(query)
    cols = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()

    df = pd.DataFrame.from_records(data = rows, columns=cols)
    df.set_index(cols[0], inplace = True)
    return df

from dotenv import load_dotenv
import os

## Connect to the SQL database using credentials from .env, and reset the database for a clean slate

In [2]:
load_dotenv()
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
DB_NAME = os.getenv("DB_NAME")

if not 'cnxn' in locals(): # type: ignore
    cnxn = pyodbc.connect("DRIVER={MySQL ODBC 9.3 Unicode Driver};"
                        "SERVER=localhost;" 
                        f"DATABASE={DB_NAME};"
                        f"USER={USER};"
                        f"PASSWORD={PASSWORD};"
                        "PORT=3306;"
                        "OPTION=3;")

cnxn.cursor().execute("DROP TABLE IF EXISTS wall_holds;")
cnxn.cursor().execute("DROP TABLE IF EXISTS walls;")
cnxn.cursor().execute("DROP TABLE IF EXISTS holds;")
cnxn.cursor().execute("DROP TABLE IF EXISTS gyms;")

cnxn.commit()

cursor = cnxn.cursor()

## Fill the holds table with preset hold names

In [3]:
# Create a table listing hold types by autogenerated ids
cursor.execute("""
                CREATE TABLE holds (
                    id SMALLINT AUTO_INCREMENT PRIMARY KEY, 
                    name VARCHAR(255) UNIQUE NOT NULL
               );
               """)
cnxn.commit()

# Fill it with some common hold names
cursor.execute("""
                INSERT INTO holds (name)
                VALUES 
                    ('Jug'),
                    ('Edge'),
                    ('Slab'),
                    ('Crimp'),
                    ('Pinch'),
                    ('Sloper'),
                    ('Undercling')
               """)
cnxn.commit()

# Run a selection to check that we've got what we expect in the table
temp_df = query_to_df("SELECT * FROM holds ORDER BY id", cursor)
temp_df.head()


,name
id,
1,Jug
2,Edge
3,Slab
4,Crimp
5,Pinch


## Create a randomized list of gyms

### Create a random list of gyms

In [4]:
import random

# Set up a list of states and associated cities to pull from 
states = ['Pennsylvania', 'New York', 'New Jersey']
cities = [['Philadelphia', 'Harrisburg', 'Pittsburgh'],['New York', 'Newark'],['Wilmington','Glassboro','Atlantic City']]
ZIP_Prefix = [[150,196],[10,14],[7,8]]

# Set up a list of randomized street name components
Street_init = ['Jane', 'Cherry', 'Dr Watson', 'Colonial', 'Center', 'Duck', 'Swan', 'New']
Street_terminal = [' Lane',' Street',' Boulevard',' Industrial Park', ' Drive']
Street_num = [0,350]

# Set up a list of randomized gym name components
Gym_prefix = ['Dave’s', 'Horton’s', 'Clear', 'Big', 'Climbathon']
Gym_terminal = [' Gym', ' Climb', ' Climbing Gym', ' Wall House']

# Generate every possible gym name from the above
names = [a + b for a in Gym_prefix for b in Gym_terminal]
gyms = []

# Associate each gym name with a random adress, city, state etc.
for name in names:
    state_idx = random.randint(0,len(states)-1) 
    state = states[state_idx]

    city = random.choice(cities[state_idx])

    zip_p = f"{random.randint(ZIP_Prefix[state_idx][0], ZIP_Prefix[state_idx][1]):03d}"
    zip_e = f"{random.randint(0,99):02d}"
    zip = zip_p + zip_e
    
    street = random.choice(Street_init) + random.choice(Street_terminal) + ' ' + str(random.randint(Street_num[0], Street_num[1]))
    
    gyms.append([name,state,city,zip,street])

random.shuffle(gyms) # Shuffle the gym list to keep all the Dave's from clumping 

del(zip)

### Create a table and fill it with the randomized gyms

In [5]:
# Create a table listing gyms that can hold all the data we wanted to add
cursor.execute("""
                CREATE TABLE gyms (
                    id SMALLINT AUTO_INCREMENT PRIMARY KEY, 
                    Name VARCHAR(255) UNIQUE NOT NULL,
                    State VARCHAR(255) NOT NULL,
                    City VARCHAR(255) NOT NULL,
                    ZipCode VARCHAR(255) NOT NULL,
                    Address VARCHAR(255) NOT NULL
               );
               """)
cnxn.commit()

# Put together a dynamic insertion string that 
insert_root = """
INSERT INTO 
    gyms (Name, State, City, ZipCode, Address)
VALUES
"""

insert = ''

for gym in gyms:
    insertion_str = ''
    for ele in gym:
        insertion_str = insertion_str + '\'' + ele + '\'' + ','

    insert = insert + '    (' + insertion_str[0:-1] + '),' + '\n'

insert = insert_root + insert[0:-2]+ ';'

# Let's show what the somewhat messily constructed insertion string looks like
print('Insertion: ', end='\n---------------------')
print(insert)

# And Run it
cnxn.execute(insert)
cnxn.commit()

# Then just like before let's check that the insertion did what we expected
print('\n\nResult:')
print('---------------------')
temp_df = query_to_df("SELECT * FROM gyms ORDER BY id", cursor)
temp_df.head()

Insertion: 
---------------------
INSERT INTO 
    gyms (Name, State, City, ZipCode, Address)
VALUES
    ('Clear Gym','New Jersey','Glassboro','00742','Duck Street 124'),
    ('Clear Wall House','New Jersey','Atlantic City','00779','Cherry Industrial Park 56'),
    ('Dave’s Climb','New Jersey','Atlantic City','00834','Dr Watson Lane 201'),
    ('Big Climbing Gym','New Jersey','Glassboro','00826','Cherry Lane 90'),
    ('Climbathon Climb','Pennsylvania','Philadelphia','18916','Cherry Drive 337'),
    ('Horton’s Gym','New York','New York','01240','Duck Drive 172'),
    ('Big Climb','Pennsylvania','Harrisburg','18694','Jane Street 271'),
    ('Climbathon Wall House','New York','Newark','01325','Center Street 20'),
    ('Horton’s Climbing Gym','New Jersey','Wilmington','00891','New Lane 190'),
    ('Dave’s Gym','Pennsylvania','Philadelphia','18994','Center Industrial Park 106'),
    ('Dave’s Climbing Gym','New Jersey','Atlantic City','00718','New Drive 333'),
    ('Clear Climb','New York',

,Name,State,City,ZipCode,Address
id,,,,,
1,Clear Gym,New Jersey,Glassboro,00742,Duck Street 124
2,Clear Wall House,New Jersey,Atlantic City,00779,Cherry Industrial Park 56
3,Dave’s Climb,New Jersey,Atlantic City,00834,Dr Watson Lane 201
4,Big Climbing Gym,New Jersey,Glassboro,00826,Cherry Lane 90
5,Climbathon Climb,Pennsylvania,Philadelphia,18916,Cherry Drive 337


## Now we can finally make some climbing-walls and associate them with gyms

### Set up the wall table, and a junction table to connect walls to holds

In [ ]:
# Set up a new table that stores walls and links them to their respective gyms.
cursor.execute("""
                CREATE TABLE walls (
                    id SMALLINT PRIMARY KEY AUTO_INCREMENT,
                    name VARCHAR(255), 
                    gym SMALLINT,
                    rating FLOAT,
                    num_ratings INT NOT NULL,
                    difficulty VARCHAR(255),
                    CHECK(rating >= 0 AND rating <= 5),
                    FOREIGN KEY (gym) REFERENCES gyms(id)
                );
               """)
cnxn.commit()

# Set up a new table that stores the holds present by wall (Junction / Graph)
cursor.execute("""
                CREATE TABLE wall_holds (
                    wall_id SMALLINT,
                    hold_id SMALLINT,
                    quantity SMALLINT NOT NULL DEFAULT 1,
                    FOREIGN KEY (hold_id) REFERENCES holds(id),
                    FOREIGN KEY (wall_id) REFERENCES walls(id),
                    PRIMARY KEY (wall_id, hold_id),
                    CHECK(quantity >= 1)
                );
               """)
cnxn.commit()


# ########################################## 
# Now we can demonstrate how a manuaL wall insertion would look before we automate it in the next cell

# Create a standardish, named, wall with a couple of ratings, a fair rating and a standard difficulty, and
# a wall in gym two that's marked as a bit harder, rated poorly, and nameless.
cursor.execute("""
                INSERT INTO walls (gym, name, rating, num_ratings, difficulty)
                VALUES
                    (1, 'Brave Blue', 3.5, 27, 'V6'),
                    (2, Null, 2.5, 1, 'V8')
               """)

# Associate some holds to the walls 
# Very vaguely wall 1 has a mix of holds, and wall two is a balancy wall with tons of slab and some crimp
cursor.execute("""
                INSERT INTO wall_holds (wall_id, hold_id, quantity)
                VALUES
                    (1, 1, 4),
                    (1, 2, 2),
                    (1, 7, 2),
                    (1, 3, 1),
                    (2, 4, 3),
                    (2, 3, 8)
               """)

# And display the result of our queries, but joined up so that we can read out the holds in a clear way.
get_walls = """
        SELECT 
               walls.id AS wall_id,
               gyms.name AS gym_name,
               walls.name AS name,
               walls.rating,
               walls.difficulty
        FROM walls
        INNER JOIN gyms ON walls.gym = gyms.id
        ORDER BY walls.rating DESC, gyms.name, walls.id
        """

print("Walls after creation of wall table:")
df = query_to_df(query, cursor)
display(df.head())

# Or if we want to know how the holds are distributed
get_holds = """
        SELECT  
                holds.name AS hold,
                CONCAT(
                    COALESCE(walls.name, 'Not Named'),
                    ' : ',
                    walls.id
                ) AS 'wall name : id',
                wall_holds.quantity AS count
        FROM 
            wall_holds
        INNER JOIN 
            holds ON holds.id = wall_holds.hold_id
        INNER JOIN
            walls ON walls.id = wall_holds.wall_id
        ORDER BY 
            walls.rating DESC, wall_id, hold
        """

print("\n\nPresence of holds on walls")
df = query_to_df(query, cursor)
display(df)

Walls after creation of wall table:


,gym_name,name,rating,difficulty
wall_id,,,,
1,Clear Gym,Brave Blue,3.5,V6
2,Clear Wall House,None,2.5,V8




Presence of holds on walls


,wall name : id,count
hold,,
Edge,Brave Blue : 1,2
Jug,Brave Blue : 1,4
Slab,Brave Blue : 1,1
Undercling,Brave Blue : 1,2
Crimp,Not Named : 2,3
Slab,Not Named : 2,8


### Randomly Generate a bunch of walls

In [ ]:
import urllib.request

words = urllib.request.urlopen("https://www.mit.edu/~ecprice/wordlist.10000").read().decode().splitlines() # https://stackoverflow.com/questions/18834636/random-word-generator-python
difficulties = ['B'] + [str(num) for num in range(0,13)]
holds = query_to_df("Select * From holds", cursor).index.tolist()

init_id = 2

def generate_wall():
    score = random.random()*5
    num_ratings = random.randint(0,1000)
    
    difficulty = 'V' + str(random.choice(difficulties))

    nameL = random.sample(words, random.randint(0,2))
    name = '-'.join([str(a) for a in nameL])
    if len(name) == 0: name = None

    hold = random.sample(holds, random.randint(1,7))
    counts = [random.randint(1,9) for h in hold]

    gymid = random.randint(1,len(gyms))

    return ([gymid, name, score, num_ratings, difficulty], [hold, counts])

num_walls = len(gyms)*6 
walls = [generate_wall() for i in range(num_walls)]

for wall in walls:
    wall_info = wall[0]
    insert_str = f"""INSERT INTO walls (gym, name, rating, num_ratings, difficulty)
                      VALUES
                         ({wall_info[0]},'{wall_info[1]}',{wall_info[2]},{wall_info[3]},'{wall_info[4]}')"""
    cursor.execute(insert_str)

    cursor.execute("SELECT LAST_INSERT_ID()")
    id = cursor.fetchone()[0]

    holds_info = wall[1]
    for hold_info in zip(holds_info[0], holds_info[1]):
            insert_str = f"""
                INSERT INTO wall_holds (wall_id, hold_id, quantity)
                VALUES
                    ({id},{hold_info[0]},{hold_info[1]})
               """
            cursor.execute(insert_str)

             
# Check the output

print("Walls after filling of wall table:")
df = query_to_df(get_walls, cursor)
display(df)

print("\n\nPresence of holds on walls")
df = query_to_df(get_holds, cursor)
display(df)

Walls after filling of wall table:


,gym_name,name,rating,difficulty
wall_id,,,,
288,Climbathon Climb,robot-shadow,4.985170,V8
312,Dave’s Wall House,None,4.971930,V6
401,Big Gym,ln-proceed,4.942390,V0
77,Clear Wall House,victorian,4.933020,V1
380,Horton’s Climb,disputes,4.930890,V8
...,...,...,...,...
145,Horton’s Climb,tied,0.053604,VB
229,Climbathon Wall House,suites,0.050540,V7
209,Climbathon Wall House,lil,0.043657,VB




Presence of holds on walls


,wall name : id,count
hold,,
Crimp,robot-shadow : 288,5
Edge,robot-shadow : 288,5
Sloper,robot-shadow : 288,3
Edge,None : 312,7
Pinch,None : 312,8
...,...,...
Edge,su : 263,8
Pinch,su : 263,9
Slab,su : 263,8
